# Welcome to the Qualification Tool for the RAPIDS Accelerator for Apache Spark
To run the tool, you need to enter a log path that represents the DBFS location for your Spark CPU event logs.  Then you can select "Run all" to execute the notebook.  After the notebook completes, you will see various output tables show up below.  More options for running the qualification tool can be found here: https://nvidia.github.io/spark-rapids/docs/spark-qualification-tool.html#qualification-tool-options.

## Summary Output
The report represents the entire app execution, including unsupported operators and non-SQL operations.  By default, the applications and queries are sorted in descending order by the following fields:
- Recommendation;
- Estimated GPU Speed-up;
- Estimated GPU Time Saved; and
- End Time.

## Stages Output
For each stage used in SQL operations, the Qualification tool generates the following information:
1. App ID
1. Stage ID
1. Average Speedup Factor: the average estimated speed-up of all the operators in the given stage.
1. Stage Task Duration: amount of time spent in tasks of SQL Dataframe operations for the given stage.
1. Unsupported Task Duration: sum of task durations for the unsupported operators. For more details, see Supported Operators.
1. Stage Estimated: True or False indicates if we had to estimate the stage duration.

## Execs Output
The Qualification tool generates a report of the “Exec” in the “SparkPlan” or “Executor Nodes” along with the estimated acceleration on the GPU. Please refer to the Supported Operators guide for more details on limitations on UDFs and unsupported operators.
1. App ID
1. SQL ID
1. Exec Name: example Filter, HashAggregate
1. Expression Name
1. Task Speedup Factor: it is simply the average acceleration of the operators based on the original CPU duration of the operator divided by the GPU duration. The tool uses historical queries and benchmarks to estimate a speed-up at an individual operator level to calculate how much a specific operator would accelerate on GPU.
1. Exec Duration: wall-Clock time measured since the operator starts till it is completed.
1. SQL Node Id
1. Exec Is Supported: whether the Exec is supported by RAPIDS or not. Please refer to the Supported Operators section.
1. Exec Stages: an array of stage IDs
1. Exec Children
1. Exec Children Node Ids
1. Exec Should Remove: whether the Op is removed from the migrated plan.

In [0]:
import json
import requests
import base64
import shlex
import subprocess
import pandas as pd

# Download Spark RAPIDS tool jar
TOOL_JAR_URL = 'https://repo1.maven.org/maven2/com/nvidia/rapids-4-spark-tools_2.12/23.04.0/rapids-4-spark-tools_2.12-23.04.0.jar'
TOOL_JAR_LOCAL_PATH = '/tmp/rapids-4-spark-tools.jar'
response = requests.get(TOOL_JAR_URL)
open(TOOL_JAR_LOCAL_PATH, "wb").write(response.content)

# Download S3 jars
HADOOP_AWS_URL = 'https://repo.maven.apache.org/maven2/org/apache/hadoop/hadoop-aws/2.7.4/hadoop-aws-2.7.4.jar'
HADOOP_AWS_LOCAL_PATH = '/tmp/hadoop-aws-2.7.4.jar'
response = requests.get(HADOOP_AWS_URL)
open(HADOOP_AWS_LOCAL_PATH, "wb").write(response.content)

AWS_JAVA_URL = 'https://repo.maven.apache.org/maven2/com/amazonaws/aws-java-sdk/1.7.4/aws-java-sdk-1.7.4.jar'
AWS_JAVA_LOCAL_PATH = '/tmp/aws-java-sdk-1.7.4.jar'
response = requests.get(AWS_JAVA_URL)
open(AWS_JAVA_LOCAL_PATH, "wb").write(response.content)



Out[38]: 11948376

In [0]:
dbutils.widgets.text("log_path", "")
eventlog_string=dbutils.widgets.get("log_path")

dbutils.widgets.text("output_path", "")
outputpath_string=dbutils.widgets.get("output_path")

In [0]:
!java -Xmx10g -cp /tmp/rapids-4-spark-tools.jar:/tmp/hadoop-aws-2.7.4.jar:/tmp/aws-java-sdk-1.7.4.jar:/databricks/jars/* com.nvidia.spark.rapids.tool.qualification.QualificationMain -o $outputpath_string $eventlog_string &> /tmp/qual_debug.log

## Summary Output

In [0]:
summary_output=pd.read_csv(outputpath_string + "/rapids_4_spark_qualification_output/rapids_4_spark_qualification_output.csv")
display(summary_output)

App Name App ID Recommendation Estimated GPU Speedup Estimated GPU Duration Estimated GPU Time Saved SQL DF Duration SQL Dataframe Task Duration App Duration GPU Opportunity Executor CPU Time Percent SQL Ids with Failures Unsupported Read File Formats and Types Unsupported Write Data Format Complex Types Nested Complex Types Potential Problems Longest SQL Duration NONSQL Task Duration Plus Overhead Unsupported Task Duration Supported SQL DF Task Duration Task Speedup Factor App Duration Estimated Unsupported Execs Unsupported Expressions TPC-DS Like Bench q14b app-20220209225233-0018 Strongly Recommended 2.7 48971.53 83400.46 112708 27066964 132372 109793 65.66 null null null null null null 112701 2939275 700037 26366927 4.16 false HashAggregate;Subquery;Execute CreateViewCommand;Filter;ReusedExchange;ColumnarToRow decimal;DecimalType TPC-DS Like Bench q14a app-20220209225013-0017 Strongly Recommended 2.66 51728.96 86381.03 117178 28746000 138110 112636 64.69 null null null null null null 117172 2978987 1114076 27631924 4.29 false HashAggregate;Subquery;Execute CreateViewCommand;Filter;ReusedExchange;ColumnarToRow decimal;DecimalType TPC-DS Like Bench q4 app-20220209224316-0007 Strongly Recommended 2.66 41856.03 69720.96 90167 85324700 111577 90167 65.09 null null null null null null 90163 3086516 0 85324700 4.41 false Execute CreateViewCommand;ReusedExchange;ColumnarToRow null TPC-DS Like Bench q24b app-20220209230348-0030 Recommended 2.34 32581.07 43720.92 56748 27574691 76302 56694 60.89 null null null null null null 56741 2950634 25998 27548693 4.37 false AdaptiveSparkPlan;Subquery;Execute CreateViewCommand;Filter;ColumnarToRow decimal TPC-DS Like Bench q67 app-20220209232913-0074 Recommended 2.27 144720.72 184416.27 308185 15437192 329137 265300 73.64 null null null null null null 308174 3028469 2148108 13289084 3.28 false Execute CreateViewCommand;ReusedExchange;HashAggregate;ColumnarToRow DecimalType TPC-DS Like Bench q24a app-20220209230232-0029 Recommended 2.25 33029.22 41483.77 53469 27911960 74513 53202 60.14 null null null null null null 53455 3305140 139193 27772767 4.54 false AdaptiveSparkPlan;Subquery;Execute CreateViewCommand;Filter;ColumnarToRow decimal TPC-DS Like Bench q93 app-20220209235239-0100 Recommended 2.18 31203.42 36894.57 50166 29213056 68098 50166 64.73 null null null null null null 50161 2973035 0 29213056 3.78 false Execute CreateViewCommand;AdaptiveSparkPlan;ColumnarToRow null TPC-DS Like Bench q23a app-20220209225917-0027 Recommended 2.17 43196.97 50617.02 74068 45913641 93814 66840 64.81 null null null null null null 74064 2975228 4480262 41433379 4.12 false HashAggregate;Subquery;Execute CreateViewCommand;Filter;ReusedExchange;ColumnarToRow decimal;DecimalType TPC-DS Like Bench q23b app-20220209230053-0028 Recommended 2.08 46588.14 50761.85 77422 59799857 97350 66526 66.43 null null null null null null 77417 3011951 8415627 51384230 4.22 false HashAggregate;Subquery;Execute CreateViewCommand;Filter;ReusedExchange;ColumnarToRow DecimalType;decimal TPC-DS Like Bench q72 app-20220209233659-0079 Recommended 1.97 30920.88 30054.11 41703 25434438 60975 41703 74.77 null null null null null null 41694 2971887 0 25434438 3.58 false Execute CreateViewCommand;ReusedExchange;ColumnarToRow null TPC-DS Like Bench q64 app-20220209232701-0071 Recommended 1.95 30544.61 29103.38 38925 31852357 59648 37301 55.12 null null null null null null 38919 2987987 1328490 30523867 4.55 false HashAggregate;Execute CreateViewCommand;Filter;ReusedExchange;ColumnarToRow decimal;DecimalType TPC-DS Like Bench q78 app-20220209234412-0085 Recommended 1.94 24390.79 22946.2 29098 24753911 47337 29098 56.99 null null null null null null 29090 2968697 0 24753911 4.73 false Execute CreateViewCommand;ReusedExchange;ColumnarToRow null TPC-DS Like Bench q95 app-20220209235427-0102 Recommended 1.9 29920.38 26967.61 37674 23337528 56888 35528 49.19 null null null null null null 37667 2958216 1328886 22008642 4.15 false Execute CreateView

## Stages Output

In [0]:
stages_output=pd.read_csv(outputpath_string + "/rapids_4_spark_qualification_output/rapids_4_spark_qualification_output_stages.csv")
display(stages_output)

App ID Stage ID Average Speedup Factor Stage Task Duration Unsupported Task Duration Stage Estimated app-20220209224509-0008 31 2.95 1184 0 false app-20220209224509-0008 33 2.9 1086 0 false app-20220209224509-0008 35 3.37 2321523 0 false app-20220209224509-0008 41 3.9 172648 0 false app-20220209224509-0008 39 5.72 4586824 0 false app-20220209224509-0008 38 3.36 694337 0 false app-20220209224509-0008 40 4.26 656835 0 false app-20220209224509-0008 37 3.36 524190 0 false app-20220209224509-0008 32 2.9 641 0 false app-20220209224509-0008 36 2.73 7065 0 false app-20220209224509-0008 42 3.0 218 0 false app-20220209224509-0008 34 2.9 1080 0 false app-20220209235114-0097 39 2.71 0 0 true app-20220209235114-0097 40 3.54 413 0 false app-20220209235114-0097 34 2.95 964 0 false app-20220209235114-0097 37 2.71 0 0 true app-20220209235114-0097 41 3.0 365 0 false app-20220209235114-0097 31 2.95 1058 0 false app-20220209235114-0097 32 2.95 694 0 false app-20220209235114-0097 35 3.68 2142147 0 false app-20220209235114-0097 38 4.34 271 0 false app-20220209235114-0097 36 3.68 457864 0 false app-20220209235114-0097 33 2.95 900 0 false app-20220209230506-0031 35 3.33 93394 0 false app-20220209230506-0031 36 3.33 239863 0 false app-20220209230506-0031 38 6.91 2696172 0 false app-20220209230506-0031 37 3.33 1684494 0 false app-20220209230506-0031 40 3.9 205018 0 false app-20220209230506-0031 32 2.9 7530 0 false app-20220209230506-0031 41 3.0 210 0 false app-20220209230506-0031 39 5.67 2129775 0 false app-20220209230506-0031 34 2.95 1198 0 false app-20220209230506-0031 31 2.95 1284 0 false app-20220209230506-0031 33 2.9 1258 0 false app-20220209232634-0070 32 2.9 1046 0 false app-20220209232634-0070 34 3.72 2226056 0 false app-20220209232634-0070 31 2.95 6677 0 false app-20220209232634-0070 33 2.95 672 0 false app-20220209232634-0070 37 3.0 210 0 false app-20220209232634-0070 35 4.3 688894 0 false app-20220209232634-0070 36 4.3 566592 113318 false app-20220209225845-0026 36 3.0 303 0 false app-20220209225845-0026 32 2.9 5690 0 false app-20220209225845-0026 31 2.95 1181 0 false app-20220209225845-0026 34 3.68 621736 0 false app-20220209225845-0026 33 2.9 894 0 false app-20220209225845-0026 35 3.9 1106260 0 false app-20220209235526-0103 34 3.68 2301205 0 false app-20220209235526-0103 31 2.95 1051 0 false app-20220209235526-0103 33 2.95 875 0 false app-20220209235526-0103 32 2.95 676 0 false app-20220209235526-0103 36 4.69 621 0 false app-20220209235526-0103 35 2.6 0 0 true app-20220209231919-0057 35 3.33 6655624 0 false app-20220209231919-0057 36 5.78 9297072 0 false app-20220209231919-0057 38 3.0 228 0 false app-20220209231919-0057 31 2.9 711 0 false app-20220209231919-0057 37 3.9 354848 0 false app-20220209231919-0057 34 3.33 134143 0 false app-20220209231919-0057 32 2.95 724 0 false app-20220209231919-0057 33 2.9 695 0 false app-20220209231533-0050 33 3.71 2489960 0 false app-20220209231533-0050 34 3.9 765614 0 false app-20220209231533-0050 35 3.0 246 0 false app-20220209231533-0050 32 2.95 1024 0 false app-20220209231533-0050 31 2.95 1096 0 false application_1665146417445_0002 4 4.5 233 0 false application_1665146417445_0002 3 4.34 2893207 0 false application_1665146417445_0002 2 2.6 14571021 7285510 false application_1665146417445_0002 1 2.0 51972027 25986013 false application_1665146417445_0002 7 4.5 244 0 false application_1665146417445_0002 6 3.98 1918109 0 false application_1665146417445_0002 5 2.6 4540826 2270413 false application_1665146417445_0002 0 1.0 4505 4505 false app-20220209230053-0028 40 3.71 4979300 0 false app-20220209230053-0028 42 3.94 517898 0 false app-20220209230053-0028 38 4.34 53 0 false app-20220209230053-0028 44 7.76 730644 0 false app-20220209230053-0028 54 3.0 221 0 false app-20220209230053-0028 35 3.33 4618718 0 false app-20220209230053-0028 36 3.33 59206 0 false app-20220209230053-0028 50 7.76 661105 0 false app-20220209230053-0028 43 3.33 25052 0 false app-20220209230053-0028 37 5.73 8697022 1449502 false app-2022020

## Execs Output

In [0]:
execs_output=pd.read_csv(outputpath_string + "/rapids_4_spark_qualification_output/rapids_4_spark_qualification_output_execs.csv")
display(execs_output)

App ID SQL ID Exec Name Expression Name Task Speedup Factor Exec Duration SQL Node Id Exec Is Supported Exec Stages Exec Children Exec Children Node Ids Exec Should Remove app-20220209224509-0008 24 ColumnarToRow null 1.0 0 30 false 31 null null true app-20220209224509-0008 24 ColumnarToRow null 1.0 0 89 false 39 null null true app-20220209224509-0008 24 WholeStageCodegen (15) WholeStageCodegen (15) 8.0 4234050 95 true 39 Sort 96 false app-20220209224509-0008 24 Exchange null 4.2 584 97 true 36:39 null null false app-20220209224509-0008 0 Execute CreateViewCommand null 1.0 0 0 false null null null false app-20220209224509-0008 24 WholeStageCodegen (8) WholeStageCodegen (8) 2.9 23059 65 true 37 Project:Filter:ColumnarToRow 66:67:68 false app-20220209224509-0008 24 Union null 3.0 0 85 true null null null false app-20220209224509-0008 24 WholeStageCodegen (22) WholeStageCodegen (22) 4.5 18839 76 true 40 HashAggregate 77 false app-20220209224509-0008 24 Project null 3.0 0 60 true null null null false app-20220209224509-0008 24 Exchange null 4.2 80029 104 true 35:39 null null false app-20220209224509-0008 24 Project null 3.0 0 40 true null null null false app-20220209224509-0008 24 Project null 3.0 0 54 true null null null false app-20220209224509-0008 24 Project null 3.0 0 81 true null null null false app-20220209224509-0008 24 Scan parquet null 3.0 536 114 true 34 null null false app-20220209224509-0008 24 WholeStageCodegen (6) WholeStageCodegen (6) 4.5 187988 9 true 40 HashAggregate 10 false app-20220209224509-0008 24 Project null 3.0 0 56 true null null null false app-20220209224509-0008 24 SubqueryBroadcast null 3.0 1736 70 true null null null false app-20220209224509-0008 14 Execute CreateViewCommand null 1.0 0 0 false null null null false app-20220209224509-0008 24 Project null 3.0 0 33 true null null null false app-20220209224509-0008 24 HashAggregate null 4.5 0 77 true 40 null null false app-20220209224509-0008 24 Scan parquet null 3.0 750661 109 true 35 null null false app-20220209224509-0008 24 HashAggregate null 4.5 0 6 true 40 null null false app-20220209224509-0008 24 Project null 3.0 0 87 true null null null false app-20220209224509-0008 10 Execute CreateViewCommand null 1.0 0 0 false null null null false app-20220209224509-0008 24 ColumnarToRow null 1.0 0 35 false 38 null null true app-20220209224509-0008 24 BroadcastExchange null 3.0 1712 38 true null null null false app-20220209224509-0008 24 WholeStageCodegen (5) WholeStageCodegen (5) 4.13 315026 12 true 38 HashAggregate:Project:BroadcastHashJoin:Project:BroadcastHashJoin 13:14:15:16:17 false app-20220209224509-0008 24 ColumnarToRow null 1.0 0 113 false 34 null null true app-20220209224509-0008 24 ColumnarToRow null 1.0 0 42 false 31 null null true app-20220209224509-0008 22 Execute CreateViewCommand null 1.0 0 0 false null null null false app-20220209224509-0008 24 WholeStageCodegen (3) WholeStageCodegen (3) 2.9 760 27 true 31 Project:Filter:ColumnarToRow 28:29:30 false app-20220209224509-0008 1 Execute CreateViewCommand null 1.0 0 0 false null null null false app-20220209224509-0008 24 BroadcastExchange null 3.0 1567 71 true null null null false app-20220209224509-0008 24 ColumnarToRow null 1.0 0 62 false 37 null null true app-20220209224509-0008 24 WholeStageCodegen (13) WholeStageCodegen (13) 2.9 17055 86 true 39 Project:Filter:ColumnarToRow 87:88:89 false app-20220209224509-0008 24 Scan parquet null 3.0 283 48 true 32 null null false app-20220209224509-0008 24 Sort null 8.0 0 103 true 39 null null false app-20220209224509-0008 23 Execute CreateViewCommand null 1.0 0 0 false null null null false app-20220209224509-0008 24 Project null 3.0 0 106 true null null null false app-20220209224509-0008 24 ColumnarToRow null 1.0 0 22 false 38 null null true app-20220209224509-0008 24 Union null 3.0 0 8 true null null null false app-20220209224509-0008 12 Execute CreateViewCommand null 1.0 0 0 false null null null false app-20220209224509-0008 24 HashAggregate null 4.5 